In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error


#Finding misclassifications

def misclassification_report(df, y_pred, y_test ):
    
    misclassified = pd.DataFrame()
    
    for count,value  in enumerate(y_test.iterrows()):
        
        if y_pred[count]!=int(value[1]['Result']):
            new_row = df.iloc[value[0]]
            new_row['Predicted'] = y_pred[count]
            misclassified = misclassified.append(new_row)
    return misclassified


In [ ]:
x_cols = ['Away_Att','Away_Def','Away_Mid','Home_Att','Home_Mid','Home_Def','Home_Form','Away_Form']
x_diff_cols = ['Att_Score','Def_Score','Mid_Score']
y_cols = ['Result']
y_regression_cols = ['Goal_Diff']




full_data = pd.read_csv('Full_Data_Form.csv' )
full_data['Goal_Diff'] = full_data['FTHG'] - full_data['FTAG']
print(full_data)

In [ ]:
x_all = full_data[x_cols]
y_all = full_data[y_regression_cols + ['Result']]
print(y_all)

In [52]:
x_train,x_test,y_train,y_test = train_test_split(x_all, y_all, test_size = 0.33, random_state = 42)


In [54]:
#Training linear regression on the goal difference 
from sklearn.linear_model import LinearRegression
import math
lr = LinearRegression()
lr.fit(x_train,y_train['Goal_Diff'])
y_pred = lr.predict(x_test[x_cols])
goals = [math.ceil(value) for value in y_pred]

pred_res = [1 if value > 0 else (-1 if value < 0 else 0) for value in goals]
z = accuracy_score(y_test['Result'],pred_res)
print(z)

0.4509283819628647


In [ ]:
#Train SVM
from sklearn.svm import LinearSVC, SVC

svm = SVC(C = 0.1, kernel = 'linear')
svm.fit(x_train,y_train)
# print(svm.coef_)
# print(svm.intercept_)
y_pred = svm.predict(x_test)
z =accuracy_score(y_test,y_pred)

#mc = misclassification_report(full_data,y_pred,y_test)
#print(mc[x_cols + y_cols + ['Predicted']])


In [ ]:
#Training decision tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3, random_state = 5)

dt.fit(x_train,y_train)

y_pred = dt.predict(x_test)

y_test_synth = [-1 if total < 0 else (1 if total > 0 else 0) for total in x_test['Total'].values] 

a = accuracy_score(y_test_synth,y_pred)
print(a)
# print(te_y.values)
mc = misclassification_report(full_data, y_pred,y_test)
print(mc)